In [47]:
import pdfplumber
import pandas as pd
import re
from tqdm import tqdm

# Step 1: Extract text from the PDF using pdfplumber
def extract_text_from_pdf(pdf_path):
    text = ""
    footer_text_1 = "*This is a system generated email. Please do not reply to this email. For further enquiry, kindly contact our customer service through https://tngd.my/careline-webform or call us at"
    footer_text_2 = "+603 5022 3888. The operating hours are Monday to Sunday, 7.00am to 10.00pm (including public holidays). Thank you."
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            # Remove the repeated footer text
            if footer_text_1 in page_text:
                page_text = page_text.split(footer_text_1)[0]
            if footer_text_2 in page_text:
                page_text = page_text.split(footer_text_2)[0]
            text += page_text + "\n"
    return text

# Step 2: Extract and parse the lines
def parse_lines(lines):
    transaction_types = ["DuitNow QR TNGD", "DuitNow QR", "Reload", "eWallet Cash Out", "DUITNOW_RECEI", "Transfer to Wallet","Payment"]
    go_transaction_types = ["GO+ Daily Earnings", "GO+ Cash In"]
    transactions = []
    go_transactions = []
    in_go_transaction_section = False
    
    for i in tqdm(range(8, len(lines))):  # Process lines starting from the 9th line
        line = lines[i]
        
        # Check if we have reached the "GO+ TRANSACTION" section
        if "GO+ TRANSACTION" in line:
            in_go_transaction_section = True
            continue
        
        # Split the line to extract the date and status
        parts = line.split(maxsplit=2)
        if len(parts) < 3:
            print(f"Skipping line due to unexpected format: {line}")
            continue
        date = parts[0]
        status = parts[1]
        remaining_line = parts[2]
        
        # Identify and remove the transaction type from the line
        if in_go_transaction_section:
            transaction_type = next((tt for tt in go_transaction_types if tt in remaining_line), "Unknown")
        else:
            transaction_type = next((tt for tt in transaction_types if tt in remaining_line), "Unknown")
        
        remaining_line = remaining_line.replace(transaction_type, "").strip()
        
        # Handle specific cases for "Reload" and "DUITNOW_RECEI"
        if transaction_type == "Reload":
            remaining_line = remaining_line.replace("Quick Reload Payment (via GO+", "Quick Reload Payment (via GO+Balance")
        elif transaction_type == "DUITNOW_RECEI":
            transaction_type = "DUITNOW_RECEIVEFROM"
            remaining_line = remaining_line.replace("DUITNOW_RECEI", "DUITNOW_RECEIVEFROM")
        
        # Initialize the reference with the first part
        reference = remaining_line.split()[0]
        description = ""
        details = ""
        
        # Split the remaining part of the line to extract the description, details, amount, and wallet balance
        parts = remaining_line.split()
        if len(parts) < 4:
            print(f"Skipping line due to unexpected format: {line}")
            continue
        wallet_balance = parts[-1]
        amount = parts[-2]
        details = parts[-3]
        description = " ".join(parts[1:-3])
        
        # Check the next line for additional reference, description, or details
        if i + 1 < len(lines):
            next_line = lines[i + 1].strip()
            print(f"Next line: {next_line}")  # Debugging statement
            parts = next_line.split()
            if "VEFROM" in next_line:
                parts.remove("VEFROM")
            if len(parts) >= 2:
                reference += " " + parts[0]
                details += " " + parts[-1]
                if "Balance)" in details:
                    details = details.replace("Balance)", "").strip()
                if len(parts) > 2:
                    description_part = " ".join(parts[1:-1])
                    if description_part:
                        description += " " + description_part
            i += 1
        
        # Keep reading lines until we reach the next date
        j = i + 1
        while j < len(lines) and not re.match(r'\d{1,2}/\d{1,2}/\d{4}', lines[j]):
            reference += " " + lines[j].strip()
            j += 1
        
        # Append the parsed parts to the appropriate transactions list
        if in_go_transaction_section:
            go_transactions.append([date, status, transaction_type, reference, description, details, amount, wallet_balance])
        else:
            transactions.append([date, status, transaction_type, reference, description, details, amount, wallet_balance])
    
    return transactions, go_transactions

# Step 3: Save the parsed data to an Excel file
def save_to_excel(transactions, go_transactions, excel_path):
    with pd.ExcelWriter(excel_path) as writer:
        df = pd.DataFrame(transactions, columns=["Date", "Status", "Transaction Type", "Reference", "Description", "Details", "Amount (RM)", "Wallet Balance"])
        df.to_excel(writer, sheet_name='Transactions', index=False)
        
        df_go = pd.DataFrame(go_transactions, columns=["Date", "Status", "Transaction Type", "Reference", "Description", "Details", "Amount (RM)", "Wallet Balance"])
        df_go.to_excel(writer, sheet_name='GO+ Transactions', index=False)

# Main function to execute the steps
def main(pdf_path, excel_path):
    text = extract_text_from_pdf(pdf_path)
    lines = text.split('\n')
    
    # Parse the lines
    transactions, go_transactions = parse_lines(lines)
    
    # Save the parsed data to an Excel file
    save_to_excel(transactions, go_transactions, excel_path)

# Example usage
pdf_path = r'C:\Users\lewka\Downloads\tng_ewallet_transactions.pdf'
excel_path = r'C:\Users\lewka\Downloads\transaction_history.xlsx'
main(pdf_path, excel_path)

100%|███████████████████████████████████████████████████████████████████████████| 2058/2058 [00:00<00:00, 97292.38it/s]

Next line: 10000010000 09144
Skipping line due to unexpected format: 10000010000 09144
Skipping line due to unexpected format: TNGOW3MY1
Skipping line due to unexpected format: 71608929442
Skipping line due to unexpected format: 143
Next line: 10000010000 Balance)
Skipping line due to unexpected format: 10000010000 Balance)
Skipping line due to unexpected format: TNGOW3MY1
Skipping line due to unexpected format: 71608929442
Skipping line due to unexpected format: 142
Next line: 10000010000 06013
Skipping line due to unexpected format: 10000010000 06013
Skipping line due to unexpected format: TNGOW3MY1
Skipping line due to unexpected format: 71608929427
Skipping line due to unexpected format: 992
Next line: 10000010000 Balance)
Skipping line due to unexpected format: 10000010000 Balance)
Skipping line due to unexpected format: TNGOW3MY1
Skipping line due to unexpected format: 71608929427
Skipping line due to unexpected format: 991
Next line: 00303358252
Skipping line due to unexpected f